In [25]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git


In [52]:
## Import
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from load_data import load_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Classifiers and kernels
import numpy as np
from numpy import reshape
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
##Loading Data
data = load_data() 
X = data
X = X.replace(np.inf, np.nan)
Y = data['label']
del X['label']

c:\Users\judit\Documents\TM1\TM10007\Groepsopdracht\TM10007\load_data.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


In [4]:
## Split
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size = 0.2, random_state = 4, stratify = Y)

In [5]:
## Features weghalen met teveel missing values
acceptabele_ratio = 0.5
train_size = len(X_train.index)
removal_rate = round(train_size*acceptabele_ratio)

X_train = X_train.dropna(axis=1, thresh=removal_rate)
common_cols = list(set(X_train.columns).intersection(X_test.columns))
X_test = X_test[common_cols]

In [75]:
## Imputation met median
X_train_missing_median = X_train.fillna(X_train.median())
X_train_missing_median = np.nan_to_num(X_train_missing_median)
X_test_missing_median = X_test.fillna(X_test.median())
X_test_missing_median = np.nan_to_num(X_test_missing_median)

In [7]:
## Scaling
scaler = preprocessing.RobustScaler()
scaler.fit(X_train_missing_median)
X_train_scaled = scaler.transform(X_train_missing_median)
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = scaler.transform(X_test_missing_median)
X_test_scaled = np.nan_to_num(X_test_scaled)

In [11]:
## PCA
pca = decomposition.PCA(n_components=0.99, svd_solver= 'full')
pca.fit(X_train_scaled)
X_train_final = pca.transform(X_train_scaled)
X_test_final = pca.transform(X_test_scaled)
explained_variance = np.cumsum(pca.explained_variance_ratio_)


In [73]:
# Construct different SVM methods
model= SVC()
model.fit(X_train_final, Y_train)

param_grid = {'degree': [1, 3, 5],
              'coef0': [0.01, 0.5, 1],
              'slack': [0.01, 0.5, 1],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ('linear', 'poly', 'rbf', 'sigmoid')}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)


Y_array = np.asarray(Y_train)
Y_train_final = Y_array.reshape((133,1))

print(X_train_final.shape)
print(Y_train_final.shape)


for X, Y in zip(X_train_final, Y_train_final):
    # Fit the grid
    grid.fit(X, Y)


# Print best parameter after tuning
print(grid.best_params_)

# Print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

# Predict with the trained model
grid_predictions = grid.predict(X_test_final)

# Print calssification report
print(classification_report(Y_test, grid_predictions))


(133, 5)
(133, 1)


ValueError: Found input variables with inconsistent numbers of samples: [5, 1]

In [ ]:
# Construct different Random Forest methods
n_estimators = [1, 5, 10, 25, 50, 75, 100]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(4, 80, 4)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
class_weight = ['balanced', 'balanced_subsample']

param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap,
              'class_weight': class_weight}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3)

# Fit the grid
grid.fit(X_train, Y_train)

# Print best parameter after tuning
print(grid.best_params_)

# Print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

# Predict with the trained model
grid_predictions = grid.predict(X_test)

# Print calssification report
print(classification_report(Y_test, grid_predictions))
